<a href="https://colab.research.google.com/github/mpagesbat/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/AG3/Marc_Pages_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



AG3. Algoritmos de optimizacion <br>
Marc Pages <br>
https://colab.research.google.com/drive/1x-b4fmiWb5Nwx4oSnmRtx7m2egtVygyz

In [0]:
!pip install requests

!pip install tsplib95

In [0]:
import urllib.request

file = "swiss42.tsp"

urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)


('swiss42.tsp', <http.client.HTTPMessage at 0x7f44793a8be0>)

In [0]:
import tsplib95
import random
import math

In [0]:
problem = tsplib95.load_problem(file)

In [0]:
Nodos=list(problem.get_nodes())
Aristas=list(problem.get_edges())


SOLUCION ALEATORIA

In [0]:
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos): 
  solucion = [0]
  for i in range(len(Nodos)-1):
    solucion = solucion + [random.choice(list(set(Nodos) - set(solucion)))]  #hacemos lista y despues set para que pueda funcionar la resta (aqui estamos calculando los nodos no elegidos)
  return solucion
# devuelve la distancia entre dos nodos

def distancia(a,b,problem):
  return problem.wfunc(a,b)

#Devuelve la distancia total de una trayectoria
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

solucion=crear_solucion(Nodos)

distancia_total(solucion,problem)
 


4892

In [0]:
def busqueda_aleatoria(problem,N):
  mejor_solucion=crear_solucion(Nodos)
  mejor_valor = distancia_total(mejor_solucion, problem)

  for i in range(N):
    solucion=crear_solucion(Nodos)
    valor = distancia_total(solucion,problem)
    if valor<mejor_valor:
      mejor_solucion=solucion
      mejor_valor=valor
  print("la mejor solucion es",mejor_valor,mejor_solucion)
busqueda_aleatoria(problem,1000)  #escogemos el numero de intentos  (normalmente da 3700, la mejor solucion encontrada es 1273)

la mejor solucion es 3802 [0, 18, 7, 34, 35, 38, 1, 32, 10, 25, 39, 22, 33, 16, 4, 3, 26, 15, 5, 13, 21, 28, 41, 2, 29, 8, 30, 12, 23, 9, 11, 24, 40, 31, 37, 17, 20, 27, 19, 14, 6, 36]


BUSQUEDA LOCAL, GENERADOR DE VECINDAD
hacemos intercambios 2-opt
total=41*40/2 =820 posiblidades de intercambios   (el 2 es para quitar las simetricas)
problema: tiende hacia minimos locales.

In [0]:
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]  #se eligen aleatoriamente 2 nodos y se intercambian
      distancia_vecina = distancia_total(vecina, problem)
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  #print(mejor_distancia,mejor_solucion)
  return mejor_solucion
      
solucion = crear_solucion(Nodos)
mejor_solucion = genera_vecina(solucion)




In [0]:
def busqueda_local(problem, N):
  #problem = datos del problema
  #N = numero de iteraciones
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  solucion_referencia = crear_solucion(Nodos)
  
  for i in range(N):
    vecina = genera_vecina(solucion_referencia)
    distancia_vecina = distancia_total(vecina, problem)
    if distancia_vecina <= mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
        
    solucion_referencia = vecina
  print("La mejor solución encontrada es " , mejor_distancia , mejor_solucion)

busqueda_local(problem,1000)

La mejor solución encontrada es  1755 [0, 13, 19, 16, 14, 5, 6, 3, 2, 28, 29, 39, 24, 40, 21, 9, 8, 23, 41, 10, 25, 11, 12, 18, 26, 4, 32, 34, 33, 20, 31, 17, 37, 15, 7, 1, 27, 30, 22, 38, 35, 36]


RECOCIDO SIMULADO


In [0]:
def genera_vecina_aleatorio(solucion):
  #Generador de 1 solucion vecina 2-opt (intercambiar 2 nodos)
  #Se puede mejorar haciendo que la elección no se uniforme sino entre las que estén más proximas
  i = random.choice(range(1, len(solucion)) )
  j = random.choice(list(set(range(1, len(solucion))) - {i}))
  if j<i:
    i,j=j,i
  vecina = list(solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:])  #se eligen aleatoriamente 2 nodos y se intercambian
  return vecina
solucion = crear_solucion(Nodos)
print(solucion,"\n",genera_vecina_aleatorio(solucion))
#print(solucion,"\n",genera_vecina_aleatorio(genera_vecina_aleatorio(solucion))) para cambiar 2 pares nodos (con recursividad)

[0, 17, 8, 25, 16, 26, 35, 12, 21, 13, 34, 1, 33, 4, 18, 37, 38, 32, 14, 39, 24, 7, 2, 41, 9, 36, 10, 6, 3, 29, 30, 19, 11, 31, 40, 15, 20, 27, 5, 28, 23, 22] 
 [0, 17, 8, 25, 16, 26, 35, 12, 21, 13, 34, 1, 33, 4, 18, 15, 38, 32, 14, 39, 24, 7, 2, 41, 9, 36, 10, 6, 3, 29, 30, 19, 11, 31, 40, 37, 20, 27, 5, 28, 23, 22]


hemos intercambiado 2 nodos

In [0]:
#Funcion de probabilidad para determinar si se cambia 
#    a una solución peor respecto a la de referencia(exponencial)
def probabilidad(T,d):
  if random.random() <= math.exp(-1*d / T)   :
    return True
  else:
    return False


def bajar_temperatura(T):
  return T*.9999    # formula descenso exponencial (podemos coger otros como (descenso constante, criterio boltzmann...))

In [0]:
#####################################################################
def recocido_simulado(problem, TEMPERATURA=1000):
  #problem = datos del problema

  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100


  while TEMPERATURA > 1:
    #Genera una solución vecina(aleatoria)
    vecina = genera_vecina_aleatorio(solucion_referencia) 
    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
    
    #Si es la mejor solución de todas se guarda
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
      print('\t****Encontrada mejor solucion:', mejor_distancia , mejor_solucion, '\n')
    
    #Si la nueva vecina es mejor se cambia y si es peor se cambia según 
    # una probabilidad dependiente de T y de |distancia_referencia - distancia_vecina|
    p = probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) )
    if distancia_vecina < distancia_referencia or p :
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina
    
    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solución encontrada es :" , mejor_distancia, mejor_solucion  )
  return mejor_solucion

sol = recocido_simulado(problem, TEMPERATURA=100000)

	****Encontrada mejor solucion: 4441 [0, 18, 25, 15, 1, 5, 29, 8, 30, 33, 21, 23, 6, 37, 36, 24, 13, 40, 35, 32, 28, 41, 39, 34, 20, 12, 10, 38, 27, 31, 14, 16, 3, 17, 2, 26, 19, 9, 11, 22, 7, 4] 

	****Encontrada mejor solucion: 4389 [0, 18, 25, 28, 1, 5, 29, 8, 30, 33, 21, 23, 6, 37, 36, 24, 2, 40, 35, 32, 15, 41, 39, 34, 20, 12, 10, 38, 27, 31, 14, 16, 3, 17, 13, 26, 19, 9, 11, 22, 7, 4] 

	****Encontrada mejor solucion: 4370 [0, 10, 9, 22, 2, 29, 24, 41, 5, 11, 39, 34, 28, 18, 6, 26, 7, 15, 4, 20, 31, 40, 12, 17, 33, 37, 36, 35, 30, 25, 16, 1, 38, 14, 32, 8, 3, 21, 27, 13, 19, 23] 

	****Encontrada mejor solucion: 4263 [0, 10, 9, 22, 2, 29, 24, 41, 5, 11, 39, 34, 28, 18, 6, 26, 7, 15, 4, 20, 31, 40, 12, 17, 33, 37, 36, 35, 30, 25, 16, 23, 38, 14, 32, 8, 3, 21, 27, 13, 19, 1] 

	****Encontrada mejor solucion: 4230 [0, 1, 31, 35, 23, 25, 38, 34, 3, 6, 15, 29, 32, 9, 36, 17, 37, 13, 20, 8, 26, 5, 39, 18, 4, 19, 11, 14, 41, 21, 24, 10, 28, 22, 33, 12, 40, 16, 7, 2, 30, 27] 

	****Encon

METODO CONSTRUCTIVO: COLONIA DE HORMIGAS

In [0]:
#primera iteracion

def Add_Nodo(problem, H ,T ) :    #poco eficiente, demasiado aleatoria
  #Establecer una una funcion de probabilidad para 
  # añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas
  Nodos = list(problem.get_nodes())
  return random.choice(   list(set(range(1,len(Nodos))) - set(H) )  )

def Incrementa_Feromona(problem, T, H):
  #Incrementar segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total 
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T

def Evaporar_Feromonas(T):
  #Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
  T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(Nodos)) ] for j in range(len(Nodos))]
  return T




def hormigas(problem, N) :
  #problem = datos del problema
  #N = Número de agentes(hormigas)
    
  #Nodos
  Nodos = list(problem.get_nodes())
    #Aristas
  Aristas = list(problem.get_edges()) 
  
  #Inicializa las aristas con una cantidad inicial de feromonas:1
  T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
  
  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]
  #Recorre cada agente construyendo la solución
  for h in range(N) :
    #print("\nAgente:", h)
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1) :
      
      #Elige el siguiente nodo
      Nuevo_Nodo = Add_Nodo(problem, Hormiga[h] ,T )
      
      Hormiga[h].append(Nuevo_Nodo)     
    
    #Incrementa feromonas en esa arista 
    T = Incrementa_Feromona(problem, T, Hormiga[h] )
    #print("Feromonas(1)", T)
      
    #Evapora Feromonas  
    T = Evaporar_Feromonas(T)
    #print("Feromonas(2)", T)

    #Seleccionamos el mejor agente
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N) :
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia =distancia_actual
  
  
  print(mejor_solucion)
  print(mejor_distancia)

hormigas(problem,1000)




  

[0, 39, 24, 23, 10, 1, 40, 11, 19, 20, 36, 16, 33, 17, 22, 38, 34, 31, 9, 2, 8, 18, 32, 4, 29, 30, 5, 27, 3, 13, 35, 28, 37, 26, 41, 21, 25, 12, 14, 15, 6, 7]
3936
